In [2611]:
import random, pprint
import simpy
import pandas as pd
from simpy.util import start_delayed

In [2612]:
class Building(object):
    def __init__(self, env, building):
        self.setAddress(building)
        self.setResident(building)
        try:
            self.setOccupancy(building)
        except AttributeError:
            print('Invalid occupancy type provided: {0}.'.format(building['occupancy']))
        self.setMonthlyCost(building)
        self.setRooms(building)
        self.setFloors(building)
        self.setAge(building)
        self.setValue(building)
        self.setDamage(building)  
        self.setInspection(env, building)
    def setAddress(self, building):
        self.address = building['address']
    def setResident(self, building):
        self.resident = building['name']
    def setOccupancy(self, building):
        self.occupancy = building['occupancy']
    def setMonthlyCost(self, building):
        self.cost = building['cost']
    def setRooms(self, building):
        self.rooms = building['rooms']
    def setFloors(self, building):
        self.floors = building['floors']
    def setAge(self, building):
        self.age = building['age']
    def setValue(self, building):
        self.value = building['value']
    def setDamage(self, building):
        self.damage = building['damage']
    def setInspection(self, env, building):
        self.inspection = env.event()
        self.inspect_start = None
        self.inspect_stop = None


In [2613]:
class Residence(Building):
    #Can verify that occupancy types only relate to residences
    def setOccupancy(self, building):
        if building['occupancy'] in ('house', 'apartment', 'condo'):
            self.occupancy = building['occupancy']
        else:
            raise AttributeError(building['occupancy'])

In [2614]:
class Household(object):
    def __init__(self, env, household):
        self.name = household['name']
        self.income = household['income']
        self.occupancy_pref = household['occupancy_pref']
        self.room_pref = household['room_pref']
        self.story = []
        self.residence = simpy.FilterStore(env) 




In [2615]:
def master_process(env, household, housing_stock):
    for household in households.keys():
        yield env.timeout(5)
        yield env.process(assign_residence(households[household]))
        yield env.process(inspect_residence(env, households[household], housing_stock))
        yield env.process(search_housing(env, households[household], housing_stock))

In [2616]:
def assign_residence(household):
    
    yield env.timeout(1)
    for residence in household.residence.items:
        household.story.append(
        '{0} lives in a {1} bedroom {2} ({3}). '.format(household.name, 
                                                        residence.rooms, 
                                                        residence.occupancy,
                                                        residence.address
                                                        )
        )

    
    

In [2617]:
# def assign_residence(household, housing_stock):
#     household.residence = yield housing_stock.get(lambda getResidence: 
#                                             getResidence.resident == household.name
#                                            )
    
#     household.story.append(
#     '{0} lives in a {1} bedroom {2} ({3}). '.format(household.name, 
#                                                     household.residence.rooms, 
#                                                     household.residence.occupancy,
#                                                     household.residence.address
#                                                     )
#     )

    
    

In [2618]:
def inspect_residence(env, household, housing_stock):  
    
    #with household.residence.items[0] as residence:
    residence = household.residence.items[0]
    residence.inspect_start = env.now

    inspection_time = 14
    yield env.timeout(inspection_time)

    residence.inspection.succeed()
    yield residence.inspection
    residence.inspection.processed

    residence.inspect_stop = env.now

    if residence.damage == 0: 
        household.story.append(
            '{0}\'s {1} ({2}) was inspected after {3} days and was green tagged. '.format(household.name, 
                                                    residence.occupancy, 
                                                    residence.address,
                                                    inspection_time)
            )

        env.exit()

    if residence.damage == 1: 
        household.story.append(
            '{0}\'s {1} ({2}) was inspected after {3} days and was yellow tagged. '.format(household.name, 
                                                    residence.occupancy, 
                                                    residence.address,
                                                    inspection_time)
            )
    if residence.damage == 2: 
        household.story.append(
            '{0}\'s {1} ({2}) was inspected after {3} days and was red tagged. '.format(household.name, 
                                                    residence.occupancy, 
                                                    residence.address,
                                                    inspection_time)
            )



In [2619]:
# def inspect_residence(env, household, housing_stock):  
    
#     household.residence.inspect_start = env.now
        
#     inspection_time = 14
#     yield env.timeout(inspection_time)
    
#     household.residence.inspection.succeed()
#     yield household.residence.inspection
    
#     household.residence.inspect_stop = env.now
    
#     if household.residence.damage == 0: 
#         household.story.append(
#             '{0}\'s {1} ({2}) was inspected after {3} days and was green tagged. '.format(household.name, 
#                                                     household.residence.occupancy, 
#                                                     household.residence.address,
#                                                     inspection_time)
#             )
        
#         env.exit()
        
#     if household.residence.damage == 1: 
#         household.story.append(
#             '{0}\'s {1} ({2}) was inspected after {3} days and was yellow tagged. '.format(household.name, 
#                                                     household.residence.occupancy, 
#                                                     household.residence.address,
#                                                     inspection_time)
#             )
#     if household.residence.damage == 2: 
#         household.story.append(
#             '{0}\'s {1} ({2}) was inspected after {3} days and was red tagged. '.format(household.name, 
#                                                     household.residence.occupancy, 
#                                                     household.residence.address,
#                                                     inspection_time)
#             )
        
#         household.residence.resident = ''
#         yield housing_stock.put(household.residence)
#         household.residence = simpy.FilterStore(env)

In [2620]:
def search_housing(env, household, housing_stock):
    residence = household.residence.items[0]
    if residence.inspection.processed and residence.damage == 2:
    #if residence.damage == 2:
        search_start = env.now

        household.story.append(
        '{0} starts search for a {1} bedroom {2} that costs less than ${3} per month on day {4}. '.format(household.name, 
                                                               household.room_pref,
                                                               household.occupancy_pref,
                                                                household.income,
                                                                            search_start)
        )

        yield env.timeout(10)



        new_residence = yield housing_stock.get(lambda getResidence: 
                                                getResidence.resident == None and
                                                getResidence.damage == 0 and
                                                getResidence.occupancy == household.occupancy_pref and 
                                                getResidence.rooms >= household.room_pref and
                                                getResidence.cost <= household.income
                                               )


        new_residence.resident = household.name
        yield household.residence.put(new_residence)

        residence.resident = None
        yield housing_stock.put(residence)



        search_stop = env.now

        household.story.append(
        '{0} occupied a ${1} per month {2} bedroom {3} ({4}) after {5} days. '.format(new_residence.resident, 
                                                               new_residence.cost,
                                                                new_residence.rooms, 
                                                               new_residence.occupancy,
                                                                new_residence.address,
                                                                search_stop)
        )

In [2621]:
# def search_housing(env, household, housing_stock):
#     #print(household.residence.inspection.processed)
#     while household.residence.inspection.processed:
# # 
#         search_start = env.now

#         household.story.append(
#         '{0} starts search for a {1} bedroom {2} that costs less than ${3} per month on day {4}. '.format(household.name, 
#                                                                household.room_pref,
#                                                                household.occupancy_pref,
#                                                                 household.income,
#                                                                             search_start)
#         )

#         yield env.timeout(10)


        
#         household.residence = yield housing_stock.get(lambda getResidence: 
#                                                 getResidence.resident == '' and
#                                                 getResidence.damage == 0 and
#                                                 getResidence.occupancy == household.occupancy_pref and 
#                                                 getResidence.rooms >= household.room_pref and
#                                                 getResidence.cost <= household.income
#                                                )

#         household.residence.resident = household.name

#         search_stop = env.now

#         household.story.append(
#         '{0} occupied a ${1} per month {2} bedroom {3} ({4}) after {5} days. '.format(household.residence.resident, 
#                                                                household.residence.cost,
#                                                                 household.residence.rooms, 
#                                                                household.residence.occupancy,
#                                                                 household.residence.address,
#                                                                 search_stop)
#         )
#     else:
#         print('No need to search')

In [2622]:
# def search_housing(env, household):
#     yield env.timeout(6)
    
#     if household.residence.damage == 0:
#         pass
#     else:
#         household.residence.resident = ''
        
#         yield housing_stock.put(household.residence)  
        
#         search_start = env.now
        
#         household.story.append(
#         '{0} starts search for a {1} bedroom {2} on day {3}. '.format(household.name, 
#                                                                household.room_pref, 
#                                                                household.occupancy_pref,
#                                                                             search_start)
#         )

#         yield env.timeout(10)

# #### vvvv Non-conditional search vvvv #####

#         household.residence = yield housing_stock.get(lambda getResidence: 
#                                                 getResidence.resident == '' and
#                                                 getResidence.damage == 0 and
#                                                 getResidence.occupancy == household.occupancy_pref and 
#                                                 getResidence.rooms >= household.room_pref and
#                                                 getResidence.cost <= household.income
#                                                )
        
#         household.residence.resident = household.name
        
#         search_stop = env.now
        
#         household.story.append(
#         '{0} occupied a {1} bedroom {2} ({3}) after {4} days. '.format(household.residence.resident, 
#                                                                household.residence.rooms, 
#                                                                household.residence.occupancy,
#                                                                 household.residence.address,
#                                                                 search_stop)
#         )
  

# ##### ^^^^^ Non-conditional search ^^^^^ ######

# ##### vvvvv TRY CONDITIONAL vvvvv ######

# #         with housing_stock.get(lambda getResidence: 
# #                                                 getResidence.resident == '' and
# #                                                 getResidence.damage == 0 and
# #                                                 getResidence.occupancy == household.occupancy_pref and 
# #                                                 getResidence.rooms >= household.room_pref and
# #                                                 getResidence.cost <= household.income
# #                                                ) as new_residence:

# #             patience = random.uniform(0, 100)
# #             # Wait for new residence or abort search
# #             search_results = yield new_residence | env.timeout(patience)

# #             if new_residence in search_results:

# #                 new_residence.resident = household.name

# #                 search_time = env.now - search_start
# #                 household.story.append(
# #                 '{0} occupied a {1} bedroom {2} ({3}) after a {4} day search. '.format(new_residence.resident, 
# #                                                                        new_residence.rooms, 
# #                                                                        new_residence.occupancy,
# #                                                                         new_residence.address,
# #                                                                         search_time)
# #                 )
                
# #             else:  
# #                 search_time = env.now = search_start

# #                 household.story.append(
# #                 '{0} gave up search after a {1} days.'.format(household.name, 
# #                                                                           search_time)
# #                 )
# ##### ^^^^^ TRY CONDITIONAL ^^^^^ ######

In [2623]:
# Manually create household input data dictionary for testing
households_inputs_dict = {'name' : ['Bill', 'Boyd', 'Bobby', 'Biff'],
                          'income' : [5000.0, 4000.0, 2000.0, 3000.0],
                        'occupancy_pref' : ['house','condo','apartment','apartment'],
                        'room_pref' : [3, 1, 1, 2],
                          'address' : ['101', '102', '103', '104'],
                          'occupancy' : ['house', 'condo', 'apartment', 'apartment'],
                        'cost' : [4000.0,3000.0,1000.0,2000.0],
                        'rooms' : [3, 1, 1, 2],
                       'floors' : [1, 1, 1, 1],
                       'age' : [1, 1, 1, 1],
                       'value' : [1, 1, 1, 1],
                       'damage' : [0, 1, 1, 2] 
                    }
# Create dataframe from dictionary
households_df = pd.DataFrame(households_inputs_dict)

# Re-order dataframe columns
#households_df = households_df[['name','income','occupancy_pref','room_pref']]
households_df

,address,age,cost,damage,floors,income,name,occupancy,occupancy_pref,room_pref,rooms,value
0,101,1,4000,0,1,5000,Bill,house,house,3,3,1
1,102,1,3000,1,1,4000,Boyd,condo,condo,1,1,1
2,103,1,1000,1,1,2000,Bobby,apartment,apartment,1,1,1
3,104,1,2000,2,1,3000,Biff,apartment,apartment,2,2,1


In [2624]:
# Manually create housing input data dictionary for testing
housing_stock_dict = {'address' : ['105', '106', '107', '108'],
                      'name' : [None, None, None, None],
                          'occupancy' : ['house', 'condo', 'apartment', 'apartment'],
                        'cost' : [4000.0,3000.0,1000.0,2000.0],
                        'rooms' : [3, 1, 1, 2],
                       'floors' : [1, 1, 1, 1],
                       'age' : [1, 1, 1, 1],
                       'value' : [1, 1, 1, 1],
                       'damage' : [0, 0, 0, 0]      
                    }
# Create dataframe from dictionary
housing_stock_df = pd.DataFrame(housing_stock_dict)

# Re-order dataframe columns
housing_stock_df

,address,age,cost,damage,floors,name,occupancy,rooms,value
0,105,1,4000,0,1,None,house,3,1
1,106,1,3000,0,1,None,condo,1,1
2,107,1,1000,0,1,None,apartment,1,1
3,108,1,2000,0,1,None,apartment,2,1


In [2625]:
env = simpy.Environment()

In [2626]:
#Instantiate residence objects
housing_stock = simpy.FilterStore(env)

for i in housing_stock_df.index:
    housing_stock.put(Residence(env, housing_stock_df.loc[i]))

for residence in housing_stock.items:
    print(residence.address,':',residence.resident)

105 : None
106 : None
107 : None
108 : None


In [2627]:
#Instantiate household objects
households = {}
for i in households_df.index:
    households[i] = Household(env, households_df.loc[i])
    households[i].residence.put(Residence(env, households_df.loc[i]))

    for residence in households[i].residence.items:
        print(residence.address,':',residence.resident)

101 : Bill
102 : Boyd
103 : Bobby
104 : Biff


In [2628]:
households[0].residence.items

In [2629]:
# for household in households.keys():
#     env.process(assign_residence(households[household]))
#     env.process(inspect_residence(env, households[household], housing_stock))
#     env.process(search_housing(env, households[household], housing_stock))

#     ah = env.process(assign_housing(env, households[household], housing_stock))
#     dh = env.process(damage_housing(env, households[household]))
#     sh = env.process(search_housing(env, households[household]))
    

#     start_delayed(env, search_housing(env, households[household], housing_stock), 2)
    



In [2630]:
env.process(master_process(env, households, housing_stock))

<Process(master_process) object at 0x109851320>

In [2631]:
env.run(until=1000)

In [2632]:
households[0].residence.items[0].inspection.processed

True

In [2633]:
for household in households.keys():
    print(''.join(households[household].story))
    print()

Bill lives in a 3 bedroom house (101). Bill's house (101) was inspected after 14 days and was green tagged. 

Boyd lives in a 1 bedroom condo (102). Boyd's condo (102) was inspected after 14 days and was yellow tagged. 

Bobby lives in a 1 bedroom apartment (103). Bobby's apartment (103) was inspected after 14 days and was yellow tagged. 

Biff lives in a 2 bedroom apartment (104). Biff's apartment (104) was inspected after 14 days and was red tagged. Biff starts search for a 2 bedroom apartment that costs less than $3000.0 per month on day 80. Biff occupied a $2000.0 per month 2 bedroom apartment (108) after 90 days. 



In [2634]:
print("There are {0} households still waiting for housing".format(len(housing_stock.get_queue)))

There are 0 households still waiting for housing


In [2635]:
housing_stock.items

In [2636]:
len(households[3].residence.items)

2

In [2637]:
bob = None